# Root square locus


The Linear Quadratic Regulator is specified by an infinite-horizon quadratic 
cost function

$$\int_0^\infty x^TQ_cx+u^TR_cu dt$$

and a linear model

$$ \dot{x}=A_cx+B_cu.$$

Under mild assumptions, the optimal policy is

$$u(t) = K_cx(t),$$

where

$$K_c = -R_c^{-1}B^TP_c,$$

and $P_c$ is the uniqute positive definite solution to

$$0 = A_c^TP_c+P_cA_c + Q_c - P_cB_cR_c^{-1}B_c^TP_c.$$

The poles of the closed loop are the eigenvalues of

$$A+BK$$

Suppose that $B \in \mathbb{R}^{n\times 1}$ and $C \in \mathbb{R}^{1\times n}$. If we pick 

$$Q_c = M^T M, R = \rho$$

we can obtain the poles by the root square locus method. This follows from the equality

$$(1-K \Phi(-s) B)(1-K \Phi(s) B))=1+\frac{1}{R}(M \Phi(-s) B)(M \Phi(s) B),$$

where $\Phi(s)=(sI-A)^{-1}$. The zeros of the transfer function on the left 
hand side correspond to the poles of the LQR closed loop and their counterparts 
on the right half plane. On the right hand side they only depend on the design 
parameters $R$ and $M$ and they can be obtained with the root locus method where

$$(M \Phi(-s) B)(M \Phi(s) B)$$

takes the role of the open-loop transfer function and $\frac{1}{R}$ takes 
the role of the loop gain. Note that this gives $2n$ poles from which the $n$ 
roots on the left half plane (stable ones) correspond to the eigenvalues of 
$A+BK$. 

The next script plots the root square locus and confirms this fact for a given 
example.

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import control
control.use_numpy_matrix(False)
%matplotlib ipympl

In [ ]:
# define system matrices
A = np.array([[0, 1, 0],[0, 0, 1],[10, 13, 2]])
B = np.array([[0],[0],[1]])

# pick M and rho
M = np.array([10, -6, 1])
rho = 1/10

# transfer functions of the root square locus method
tf1 = control.ss2tf(A,B,M,0)
num1 = tf1.num[0][0]
den1 = tf1.den[0][0]
tf2 = control.ss2tf(-A,B,-M,0)
num2 = tf2.num[0][0]
den2 = tf2.den[0][0]
tf3 = 1+1/rho*tf1*tf2
Z = signal.tf2zpk(tf3.num[0][0], tf3.num[0][0])[0]

# obtain gains in the traditional way
K,S,E = control.lqr(A,B,M.reshape(3,1)@M.reshape(1,3),rho)
K = -K
Z2 = np.linalg.eig(A+B@K)[0]
Z2

In [ ]:
# root square locus
fig = plt.figure()
control.rlocus(tf1*tf2);

In [ ]:
# compare the location of the poles for the two methods
fig = plt.figure()
ax = plt.gca()
plt.plot(Z.real, Z.imag, 'x')
plt.plot(Z2.real, Z2.imag, 'o', fillstyle='none')
ax.grid(True)
ax.set_xlabel('Re(z)')
ax.set_ylabel('Im(z)')
ax.set_title('Location of closed loop poles')
ax.legend([r'zeros of $1+1\rho M*(sI-A)^{-1}BM*(-sI-A)^{-1}B$',r'$eig(A+B*K)$']);